In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import time

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
start_time = time.time()
nlp = spacy.load('en')
alice_doc = nlp(alice)
interval = (time.time() - start_time) / 60
print('Time taken: {} minutes.'.format('%.1f' % interval))

Time taken: 0.2 minutes.


In [4]:
start_time = time.time()
persuasion_doc = nlp(persuasion)
interval = (time.time() - start_time) / 60
print('Time taken: {} minutes.'.format('%.1f' % interval))

Time taken: 0.5 minutes.


In [7]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [8]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
   
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)    

In [9]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,memory,hopeful,shin,assure,wine,despair,whisper,emphasis,simple,choose,...,grace,kick,attachment,beautiful,unfortunate,refreshment,degree,tree,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [10]:
# Sentence length, POS, repeat words, avg word length

# Sentence length:
word_counts['sentence_length'] = [len(sentence) for sentence in word_counts['text_sentence']]
                                  
word_counts['pos_list'] = [[token.pos_ for token in word_counts['text_sentence'][i]] 
                           for i in range(0, len(word_counts['text_sentence']))]
                                                       
pos = ['ADV', 'VERB', 'NOUN', 'ADP', 'PROPN', 'ADJ', 'DET', 'PUNCT', 'PRON', 'PART']

# No. of each POS contained in the sentence:                                  
for item in pos:
    word_counts[item] = [word_counts['pos_list'][i].count(item) for i in range(0, len(word_counts['pos_list']))]                                  

    
    
# Mean word length                                  
word_counts['word_length'] = [np.mean([len(token) for token in word_counts['text_sentence'][i]]) for i in range(0, len(word_counts['text_sentence']))]
                                  
                                  


In [11]:
word_counts.iloc[:, -13:].head()

,sentence_length,pos_list,ADV,VERB,NOUN,ADP,PROPN,ADJ,DET,PUNCT,PRON,PART,word_length
0,67,"[PROPN, VERB, VERB, PART, VERB, ADV, ADJ, ADP,...",3,13,12,8,2,3,5,10,3,2,3.656716
1,63,"[ADV, PRON, VERB, VERB, ADP, ADJ, ADJ, NOUN, P...",7,11,8,8,2,7,6,7,4,1,3.730159
2,33,"[ADV, VERB, NOUN, ADV, ADV, ADJ, ADP, DET, PUN...",6,5,2,4,2,1,3,4,2,1,3.393939
3,3,"[INTJ, INTJ, PUNCT]",0,0,0,0,0,0,0,1,0,0,2.333333
4,6,"[PRON, VERB, VERB, ADJ, PUNCT, PUNCT]",0,2,0,0,0,1,0,2,1,0,2.333333


In [12]:
word_counts.loc[0]['text_sentence']

Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversation?'

In [13]:
n = 346

In [14]:
phrase = word_counts.loc[n]['text_sentence']

In [15]:
pos_phrase = word_counts.loc[n]['pos_list']

In [16]:
word_tuples = [(phrase[i], pos_phrase[i]) for i in range(0, len(phrase))]

In [17]:
table = pd.DataFrame(word_tuples, columns = ['Word', 'POS'])
table#[table['POS'] == 'ADJ']

,Word,POS
0,How,ADV
1,surprised,ADJ
2,he,PRON
3,'ll,VERB
4,be,VERB
5,when,ADV
6,he,PRON
7,finds,VERB
8,out,PART
9,who,NOUN


### First model only the dummies

In [18]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

start_time = time.time()
rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
to_drop = ['text_sentence','text_source', 'sentence_length', 'pos_list', 'word_length']
to_drop.extend(pos)
X = np.array(word_counts.drop(to_drop, 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))
interval = (time.time() - start_time) / 60
print('\Time taken: {} minutes.'.format('%.1f' % interval))

Training set score: 0.987774294671

Test set score: 0.886748120301
\Time taken: 0.1 minutes.


In [19]:
from sklearn.linear_model import LogisticRegression

start_time = time.time()
lr = LogisticRegression()
train = lr.fit(X_train, y_train)


print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))
interval = (time.time() - start_time) / 60
print('Time taken: {} minutes.'.format('%.1f' % interval))

Training set score: 0.957993730408

Test set score: 0.915883458647
Time taken: 0.0 minutes.


In [20]:
clf = ensemble.GradientBoostingClassifier()

start_time = time.time()
train = clf.fit(X_train, y_train)


print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))
interval = (time.time() - start_time) / 60
print('Time taken: {} minutes.'.format('%.1f' % interval))

Training set score: 0.886833855799

Test set score: 0.873590225564
Time taken: 0.9 minutes.


### Then model the dummies plus new variables
As can be seen below, the addition of new features has not done much to improve the accuracy of the modeling.

#### Random Forest Classifier

In [21]:
start_time = time.time()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source', 'pos_list'], 1))
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))
interval = (time.time() - start_time) / 60
print('Time taken: {} minutes.'.format('%.1f' % interval))

Training set score: 0.991222570533

Test set score: 0.859962406015
Time taken: 0.1 minutes.


#### Logistic (Ridge) Regression

In [22]:
start_time = time.time()
lr = LogisticRegression()
train = lr.fit(X_train, y_train)


print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))
interval = (time.time() - start_time) / 60
print('Time taken: {} minutes.'.format('%.1f' % interval))

Training set score: 0.960188087774

Test set score: 0.915413533835
Time taken: 0.0 minutes.


#### Gradient Boosting

In [23]:
clf = ensemble.GradientBoostingClassifier()

sh
train = clf.fit(X_train, y_train)


print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))
interval = (time.time() - start_time) / 60
print('Time taken: {} minutes.'.format('%.1f' % interval))

Training set score: 0.898119122257

Test set score: 0.882518796992
Time taken: 0.8 minutes.


#### SVC

In [24]:
from sklearn.svm import SVC
svc = SVC()
start_time = time.time()
train = svc.fit(X_train, y_train)

print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))
interval = (time.time() - start_time) / 60
print('Time taken: {} minutes.'.format('%.1f' % interval))

Training set score: 0.697492163009

Test set score: 0.708176691729
Time taken: 1.4 minutes.


### Test the model against other texts
As the results (above) were no better with the added variables, we will remove the extra variables and model the texts as before, using only the bag of words data. We change the code below according to the texts being compared:

In [25]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [26]:
persuasion[:100]

'Sir Walter Elliot, of Kellynch Hall, in Somersetshire, was a man who, for his own amusement, never t'

In [27]:
alice = gutenberg.raw('carroll-alice.txt')

In [28]:
alice[:100]

"[Alice's Adventures in Wonderland by Lewis Carroll 1865]\n\nCHAPTER I. Down the Rabbit-Hole\n\nAlice was"

In [29]:
persuasion = gutenberg.raw('austen-persuasion.txt')
persuasion[:100]

'[Persuasion by Jane Austen 1818]\n\n\nChapter 1\n\n\nSir Walter Elliot, of Kellynch Hall, in Somersetshire'

In [30]:
alice = re.sub(r'CHAPTER .*', '', alice)
alice[:100]

"[Alice's Adventures in Wonderland by Lewis Carroll 1865]\n\n\n\nAlice was beginning to get very tired of"

In [31]:
persuasion = re.sub(r'Chapter \d+', '', persuasion)
persuasion[:100]

'[Persuasion by Jane Austen 1818]\n\n\n\n\n\nSir Walter Elliot, of Kellynch Hall, in Somersetshire, was a m'

In [72]:
ball = gutenberg.raw('chesterton-ball.txt')
#moby[:200]

In [73]:
# There are no uses of the word 'Chapter':
ball[0:100]

'[The Ball and The Cross by G.K. Chesterton 1909]\n\n\nI. A DISCUSSION SOMEWHAT IN THE AIR\n\nThe flying s'

In [74]:
ball = text_cleaner(ball)

In [ ]:
# Parse the cleaned novel.
start_time = time.time()
ball_doc = nlp(ball)
interval = (time.time() - start_time) / 60
print('Time taken: {} minutes.'.format('%.1f' % interval))
print(len(ball))

In [76]:
thursday = gutenberg.raw('chesterton-thursday.txt')

In [79]:
# There are no uses of the word 'Chapter':
thursday[0:100]

'[The Man Who Was Thursday by G. K. Chesterton 1908]\n\nTo Edmund Clerihew Bentley\n\nA cloud was on the '

In [80]:
thursday = text_cleaner(thursday)

In [81]:
# Parse the cleaned novel.
start_time = time.time()
thursday_doc = nlp(thursday)
interval = (time.time() - start_time) / 60
print('Time taken: {} minutes.'.format('%.1f' % interval))
print(len(thursday))

Time taken: 0.3 minutes.
318836


#### Having removed all chapter headings, we can start the analysis.

In [90]:
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
ball_sents = [[sent, "Chesterton"] for sent in ball_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(persuasion_sents + ball_sents)
sentences.head()

,0,1
0,"(Sir, Walter, Elliot, ,, of, Kellynch, Hall, ,...",Austen
1,"(This, was, the, page, at, which, the, favouri...",Austen
2,"(Walter, Elliot, ,, born, March, 1, ,, 1760, ,...",Austen
3,"(of, South, Park, ,, in, the, county, of, Glou...",Austen
4,"("", Precisely, such, had, the, paragraph, orig...",Austen


In [91]:
# Set up the bags.
alicewords = bag_of_words(alice_doc)
ballwords = bag_of_words(ball_doc)
thursdaywords = bag_of_words(thursday_doc)
# Combine bags to create a set of unique words.
common_words = set(persuasionwords + ballwords) 

In [92]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000
Processing row 5500
Processing row 6000
Processing row 6500
Processing row 7000
Processing row 7500
Processing row 8000


,level,coincidence,gaiety,assure,memory,shin,stride,wine,whisper,pistol,...,attachment,kick,sweep,beautiful,unfortunate,degree,tree,contradiction,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Sir, Walter, Elliot, ,, of, Kellynch, Hall, ,...",Austen
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(This, was, the, page, at, which, the, favouri...",Austen
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Walter, Elliot, ,, born, March, 1, ,, 1760, ,...",Austen
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(of, South, Park, ,, in, the, county, of, Glou...",Austen
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"("", Precisely, such, had, the, paragraph, orig...",Austen


#### Random Forest Classifier

In [94]:
start_time = time.time()
rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
#to_drop = ['text_sentence','text_source', 'sentence_length', 'pos_list', 'word_length']
#to_drop.extend(pos)
X = np.array(word_counts.drop(['text_source', 'text_sentence'], axis=1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))
interval = (time.time() - start_time) / 60
print('\Time taken: {} minutes.'.format('%.1f' % interval))

Training set score: 0.984948826008

Test set score: 0.833232992173
\Time taken: 0.1 minutes.


#### Logistic (Ridge) Regression

In [95]:
start_time = time.time()
lr = LogisticRegression()
train = lr.fit(X_train, y_train)

print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))
interval = (time.time() - start_time) / 60
print('Time taken: {} minutes.'.format('%.1f' % interval))

Training set score: 0.951434878587

Test set score: 0.879590608067
Time taken: 0.0 minutes.


#### Gradient Boosting Classifier

In [96]:
clf = ensemble.GradientBoostingClassifier()

start_time = time.time()
train = clf.fit(X_train, y_train)


print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))
interval = (time.time() - start_time) / 60
print('Time taken: {} minutes.'.format('%.1f' % interval))

Training set score: 0.807947019868

Test set score: 0.792895845876
Time taken: 1.7 minutes.


## Results
-  The Support Vector Classifier was the worst performer among the models used, which is why it was not used after the first round. 
-  Of the remaining models, Logistic (Ridge) Regression gave the highest accuracy scores in the test set. 
-  Gradient Boosting, while not the most accurate, seemed to give the most consistency between test and training sets.  
-  Test-set accuracy was uniformly lower when comparing two works by the same author (The Man Who Was Thursday and The Ball and The Cross, by G.K. Chesterton).

#### Persuasion vs. Alice in Wonderland 
Train / Test:

RFC: 0.991/0.860    <BR/>
Logistic (Ridge) Regression: 0.960/0.915      <BR/>
Gradient Boosting:  0.899/0.883                <BR/>
Support Vector Classifier: 0.697/0.708

#### Alice in Wonderland vs. The Ball and the Cross
Train / Test:

RFC:  0.986/0.855      <BR/>
Logistic (Ridge) Regression:   0.945/0.901      <BR/>
Gradient Boosting:   0.878/0.867

#### Persuasion vs. The Ball and the Cross
Train / Test:

RFC: 0.985/0.833     <BR/>
Logistic (Ridge) Regression: 0.951/0.880      <BR/>
Gradient Boosting: 0.808/0.793

#### The Man Who Was Thursday vs. The Ball and the Cross
Train / Test:

RFC: 0.976/0.733     <BR/>
Logistic (Ridge) Regression: 0.912/0.771      <BR/>
Gradient Boosting: 0.751/0.724